In [ ]:
from pythonnet import load
load("coreclr")

import os
import clr
clr.AddReference(os.path.abspath("../src/DerivaSharp/bin/Release/net9.0/win-x64/publish/DerivaSharp.dll"))

import matplotlib.pyplot as plt
import numpy as np

from DerivaSharp.Instruments import BarrierTouchStatus, SnowballOption
from DerivaSharp.PricingEngines import FdSnowballEngine, FiniteDifferenceScheme, McSnowballEngine, PricingContext
from DerivaSharp.Time import DateUtils
from System import DateOnly

In [ ]:
effectiveDate = DateOnly(2022, 1, 5)
expirationDate = DateOnly(2023, 1, 5)
lockUpMonths = 3
koObsDates = list(DateUtils.GetObservationDates(effectiveDate, expirationDate, lockUpMonths))

for date in koObsDates:
    print(date.ToString())

In [ ]:
koCoupon = 0.085
matCoupon = 0.085
ki = 0.8
ko = 1.03
strike = 1.0
vol = 0.16
r = 0.02
q = 0.04

option = SnowballOption.CreateStandardSnowball(
    0.0845,
    1.0,
    0.8,
    1.03,
    koObsDates,
    BarrierTouchStatus.NoTouch,
    effectiveDate,
    expirationDate,
)

fd_engine = FdSnowballEngine(FiniteDifferenceScheme.CrankNicolson, 1000, 500)
mc_engine = McSnowballEngine(100000)

In [ ]:
valuationDate = DateOnly(2022, 1, 5)
spots = np.arange(0.5, 1.51, 0.01, dtype=np.float64)
ctx = PricingContext(1, valuationDate, vol, r, q)

fd_values = -np.array(fd_engine.Values(option, ctx, spots))
fd_deltas = -np.array(fd_engine.Deltas(option, ctx, spots))
fd_gammas = -np.array(fd_engine.Gammas(option, ctx, spots))

mc_values = -np.array(mc_engine.Values(option, ctx, spots))
mc_deltas = -np.array(mc_engine.Deltas(option, ctx, spots))
mc_gammas = -np.array(mc_engine.Gammas(option, ctx, spots))

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(4, 9))

axs[0].plot(spots, fd_values, label="FD")
axs[0].plot(spots, mc_values, label="MC")
axs[0].set_title("Value")
axs[0].grid(True)
axs[0].legend()

axs[1].plot(spots, fd_deltas, label="FD")
axs[1].plot(spots, mc_deltas, label="MC")
axs[1].set_title("Delta")
axs[1].grid(True)
axs[1].legend()

axs[2].plot(spots, fd_gammas, label="FD")
axs[2].plot(spots, mc_gammas, label="MC")
axs[2].set_title("Gamma")
axs[2].grid(True)
axs[2].legend()

plt.tight_layout()
plt.show()